# Postprocess predictions and create a submission

In [1]:
#@title Submission notes and params { run: "auto" }
version_notes = "BBox reduction" #@param {type:"string"}

reduce_method = "weighted_boxes_fusion" #@param ["None", "nms", "soft_nms", "non_maximum_weighted", "weighted_boxes_fusion"]

iou_threshold = 0.6 #@param {type:"slider", min:0, max:1, step:0.01}
skip_box_thrreshold = 0.1 #@param {type:"slider", min:0, max:1, step:0.01}
sigma = 0.1 #@param {type:"slider", min:0, max:1, step:0.01}

In [2]:
#@title Select models versions { run: "auto" }

binary_version = "4: EffNet, 1024 resolution, threshold 0.8" #@param ["2: Try 1024 resolution", "3: EffNet, 1024 resolution, threshold 0.8", "4: EffNet, 1024 resolution, threshold 0.8", "5: EffNet, 512 resolution, threshold 0.8"]
binary_notes = binary_version.split(":")[1]
binary_version = int(binary_version.split(":")[0])

yolo_version_5_selected = False #@param {type: "boolean"}
yolo_version_6_selected = True #@param {type: "boolean"}
yolo_version_7_selected = True #@param {type: "boolean"}
yolo_version_8_selected = True #@param {type: "boolean"}
yolo_version_9_selected = True #@param {type: "boolean"}
yolo_version_10_selected = True #@param {type: "boolean"}
yolo_version_11_selected = False #@param {type: "boolean"}

In [3]:
INPUTS_YOLO_VERSIONS = [
    {
        "version": 5,
        "version_notes": "YOLOv5x, 50 epochs, random rad, 20% valid split, 1024 size, IOU 0.35, conf 0.15",
        "path": "v5",
        "enabled": yolo_version_5_selected,
    },
    {
        "version": 6,
        "version_notes": "WBF preproc, YOLOv5x, 50 epochs, 20% valid split, 1024 size, IOU 0.35, conf 0.15",
        "path": "v6",
        "enabled": yolo_version_6_selected,
    },
    {
        "version": 7,
        "version_notes": "WBF preproc, YOLOv5x, 40 epochs, 20% valid split, 1024 size, IOU 0.6, conf 0.1",
        "path": "v7",
        "enabled": yolo_version_7_selected,
    },
    {
        "version": 8,
        "version_notes": "WBF preproc with IoU 0.4, YOLOv5x, 40 epochs, 20% valid split, 1024 size, IOU 0.35, conf 0.15, remove (most) augs",
        "path": "v8",
        "enabled": yolo_version_8_selected,
    },
    {
        "version": 9,
        "version_notes": "WBF preproc with IoU 0.7, YOLOv5x, 40 epochs, 20% valid split, 1024 size, IOU 0.6, conf 0.1, remove (most) augs",
        "path": "v9",
        "enabled": yolo_version_9_selected,
    },
    {
        "version": 10,
        "version_notes": "WBF preproc with IoU 0.7, YOLOv5x, 25 epochs, 20% valid split, 1024 size, IOU 0.6, conf 0.1, remove (most) augs",
        "path": "v10",
        "enabled": yolo_version_10_selected,
    },
    {
        "version": 11,
        "version_notes": "Remove (most) augs --- WBF preproc IoU: 0.8, version: yolov5l, epochs: 30, valid split: 20.0%, image size: 1024, detect IoU: 0.8, detect conf: 0.05",
        "path": "v11",
        "enabled": yolo_version_11_selected,
    },
]

# Check.
# Check for errors in fields.
assert(all(all(key_name in yolo_version for key_name in ["version", "version_notes", "path", "enabled"]) for yolo_version in INPUTS_YOLO_VERSIONS))
# Check that versions are distinct (no need to ensemble the same model)
assert(len(set(yolo_version["version"] for yolo_version in INPUTS_YOLO_VERSIONS)) == len(INPUTS_YOLO_VERSIONS))
# Check that paths are distinct.
assert(len(set(yolo_version["path"] for yolo_version in INPUTS_YOLO_VERSIONS)) == len(INPUTS_YOLO_VERSIONS))

# Filter only enabled.
INPUTS_YOLO_VERSIONS = [yolo_version for yolo_version in INPUTS_YOLO_VERSIONS if yolo_version["enabled"]]

## Setup

In [4]:
try:
    from google.colab import drive
    drive.mount("/content/drive")
    %cd /content/drive/MyDrive/Colab\ Notebooks/kaggle
    from setup_colab import setup_colab_for_kaggle, WORK_FOLDER
    setup_colab_for_kaggle(check_env=False, local_working=True)
except:
    print("Not in Colab")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Colab Notebooks/kaggle
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Content of Drive Kaggle data dir (/content/drive/MyDrive/kaggle): ['/content/drive/MyDrive/kaggle/input', '/content/drive/MyDrive/kaggle/working', '/content/drive/MyDrive/kaggle/.ipynb_checkpoints', '/content/drive/MyDrive/kaggle/output']
Content of Kaggle data dir (/kaggle): ['/kaggle/working', '/kaggle/input', '/kaggle/output']
Content of Kaggle data subdir (/kaggle/input): ['/kaggle/input/cassava-model', '/kaggle/input/cassava-leaf-disease-classification', '/kaggle/input/googlebitemperedloss', '/kaggle/input/vbdyolo', '/kaggle/input/.ipynb_checkpoints', '/kaggle/input/vinbigdata', '/kaggle/input/vinbigdata-chest-xray-abnormalities-detection', '/kaggle/input/vinbigdata-chest-xray-origi

In [5]:
from pathlib import Path

from IPython.display import clear_output

import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

INPUT_FOLDER_ORIGINAL_PNG = WORK_FOLDER / "vinbigdata-chest-xray-original-png"
INPUT_FOLDER_YOLO_OUT = WORK_FOLDER / "vbdyolo-out"
INPUT_FOLDER_BINARY = WORK_FOLDER / "vbdbinary"

for yolo_version in INPUTS_YOLO_VERSIONS:
    yolo_version["path"] = INPUT_FOLDER_YOLO_OUT / yolo_version["path"]

print(f"Using the following YOLO versions:")
print("\n".join(str(yolo_version) for yolo_version in INPUTS_YOLO_VERSIONS))

Using the following YOLO versions:
{'version': 11, 'version_notes': 'Remove (most) augs --- WBF preproc IoU: 0.8, version: yolov5l, epochs: 30, valid split: 20.0%, image size: 1024, detect IoU: 0.8, detect conf: 0.05', 'path': PosixPath('/kaggle/working/vbdyolo-out/v11'), 'enabled': True}


## Get data from Kaggle

In [6]:
!pip install -U git+https://github.com/Witalia008/kaggle-api.git@fix-datasets-download-file-unzip
clear_output()

In [7]:
!kaggle datasets download corochann/vinbigdata-chest-xray-original-png -f test_meta.csv -p {INPUT_FOLDER_ORIGINAL_PNG} --unzip

  0% 0.00/126k [00:00<?, ?B/s]
100% 126k/126k [00:00<00:00, 48.3MB/s]


In [8]:
!pip install -U git+https://github.com/Witalia008/kaggle-api.git@add-datasets-download-version
clear_output()

In [9]:
for yolo_version in INPUTS_YOLO_VERSIONS:
    !rm -rf {yolo_version["path"]}
    !kaggle datasets download "witalia/vbdyolo-out-newest" -v {yolo_version["version"]} -p {yolo_version["path"]} --unzip --force

 72% 60.0M/83.9M [00:00<00:00, 289MB/s]
100% 83.9M/83.9M [00:00<00:00, 325MB/s]


In [10]:
!rm -rf {INPUT_FOLDER_BINARY}
!kaggle datasets download "witalia/vbdbinary" -v {binary_version} -p {INPUT_FOLDER_BINARY} --unzip --force

 83% 86.0M/104M [00:00<00:00, 291MB/s]
100% 104M/104M [00:00<00:00, 254MB/s] 


## Process YOLO output

### BBox reduction (potentially from multiple models)

In [11]:
!pip install ensemble-boxes
clear_output()

In [12]:
from ensemble_boxes import *
from typing import List

def reduce_boxes(image_labels_list: List[pd.DataFrame], iou_thr=iou_threshold, skip_box_thr=skip_box_thrreshold, sigma=sigma, method=None):
    label_empty = [len(df) == 0 for df in image_labels_list]

    if len(image_labels_list) == 0 or all(label_empty):
        # If there's nothing to fuse, return empty.
        return pd.DataFrame(columns=["class_id", "conf", "x_min", "y_min", "x_max", "y_max"])

    # Ensemble_boxes doesn't do well with empty inputs, so skip those.
    labels = [image_labels["class_id"].values for image_labels, empty in zip(image_labels_list, label_empty) if not empty]
    scores = [image_labels["conf"].values for image_labels, empty in zip(image_labels_list, label_empty) if not empty]
    boxes = [image_labels[["x_min", "y_min", "x_max", "y_max"]].values for image_labels, empty in zip(image_labels_list, label_empty) if not empty]

    if method == non_maximum_weighted or method == weighted_boxes_fusion:
        boxes, scores, labels = method(boxes, scores, labels, iou_thr=iou_thr, skip_box_thr=skip_box_thr)
    elif method == soft_nms:
        boxes, scores, labels = method(boxes, scores, labels, iou_thr=iou_thr, thresh=skip_box_thr, sigma=sigma)
    elif method == nms:
        boxes, scores, labels = method(boxes, scores, labels, iou_thr=iou_thr)
    else:
        if len(labels) > 1:
            raise ValueError("Cannot fuse more than 1 model without a fusing method")
        [boxes], [scores], [labels] = boxes, scores, labels

    reduced_labels = pd.DataFrame().reindex(list(range(len(labels))))
    reduced_labels["class_id"] = np.array(labels).astype(np.int32)
    reduced_labels["conf"] = scores
    reduced_labels[["x_min", "y_min", "x_max", "y_max"]] = boxes

    return reduced_labels

### Read and transform labels

In [13]:
def read_prediction_labels(filename: Path):
    yolo_columns = ["class_id", "x_centre", "y_centre", "bw", "bh", "conf"]
    if filename.exists():
        labels: pd.DataFrame = pd.read_csv(filename, delimiter=" ", header=None)
        labels.columns = yolo_columns
    else:
        labels: pd.DataFrame = pd.DataFrame(columns=yolo_columns)

    # Convert YOLO format (x_centre, y_centre, bw, bh) to competition format (x_min, y_min, x_max, y_max)
    labels["x_min"] = (labels["x_centre"] - labels["bw"] / 2).clip(0, 1)
    labels["y_min"] = (labels["y_centre"] - labels["bh"] / 2).clip(0, 1)
    labels["x_max"] = (labels["x_centre"] + labels["bw"] / 2).clip(0, 1)
    labels["y_max"] = (labels["y_centre"] + labels["bh"] / 2).clip(0, 1)
    labels = labels.drop(columns=["x_centre", "y_centre", "bw", "bh"])
    # After dropping, conf column should become the second one.
    assert(labels.columns.to_list() == ["class_id", "conf", "x_min", "y_min", "x_max", "y_max"])

    return labels


def scale_labels(labels: pd.DataFrame, image_w: int, image_h: int):
    # Scale coordinates to image's size. Clip to make sure it's not out of bounds of the image.
    labels[["x_min", "x_max"]] = (labels[["x_min", "x_max"]] * image_w).round().astype(np.int32).clip(0, image_w - 1)
    labels[["y_min", "y_max"]] = (labels[["y_min", "y_max"]] * image_h).round().astype(np.int32).clip(0, image_h - 1)

    return labels


def labels_to_string(labels: pd.DataFrame):
    # Empty DataFrame means there was no YOLO output file or "reduce" removed all bboxes.
    # Means "No finding"
    if len(labels) == 0:
        return "14 1 0 0 1 1"
    # Convert all rows to one prediction string
    return " ".join(labels.to_string(header=False, index=False).split())

### Process YOLO output files, and calculate the predictions

In [14]:
import ensemble_boxes

results_yolo_df = pd.DataFrame(columns=["image_id", "PredictionString"])

test_metadata = pd.read_csv(INPUT_FOLDER_ORIGINAL_PNG / "test_meta.csv")
test_metadata = test_metadata.set_index("image_id").to_dict("index")

for image_id, image_dims in tqdm(test_metadata.items(), total=len(test_metadata)):
    # Read YOLO output to pandas DataFrame.
    prediction_labels = [read_prediction_labels(
        yolo_version["path"] / "labels_pred" / f"{image_id}.txt"
    ) for yolo_version in INPUTS_YOLO_VERSIONS]

    # Reduce bboxes.
    method = None if reduce_method == "None" else getattr(ensemble_boxes, reduce_method)
    prediction_labels = reduce_boxes(prediction_labels, method=method)

    # Scale them to the original size.
    # NOTE: dim0 and dim1 are reversed: y-axis, x-axis!
    prediction_labels = scale_labels(prediction_labels, image_dims["dim1"], image_dims["dim0"])

    # Convert resulting bboxes from DataFrame to string.
    prediction_str = labels_to_string(prediction_labels)

    results_yolo_df = results_yolo_df.append({"image_id": image_id, "PredictionString": prediction_str}, ignore_index=True)

results_yolo_df.sample(10, random_state=10)

,image_id,PredictionString
1779,9b41c13ac918adcc1261f750a1d61b40,11 0.371154 769 1953 833 2009 8 0.330505 420 1...
341,202dc97c4b3ec7e50f4813456938500a,3 0.838379 703 1177 1560 1531 0 0.488403 965 6...
1276,711dc542c6c7e2b21889838cd050a810,14 1 0 0 1 1
1012,5ba0a669fd3430e9386758e838210b98,11 0.352539 972 966 1161 1182 0 0.316895 1782 ...
470,2b2810284313c5547241b38668f67b2c,8 0.462891 389 1972 452 2022 0 0.396322 1079 9...
1755,990b0dcd83353894c4a2e73738cd6c25,14 1 0 0 1 1
2773,ec6c4e81b448275577d3e4a74acbe443,0 0.466065 1101 691 1315 916 3 0.464844 851 13...
1085,6100bc75e0a668c382d92d5eb9038076,3 0.346842 664 1119 1550 1402
2055,b1a596318e7bfdcbb74044bf6d38c6ed,3 0.657227 643 1258 1431 1564 0 0.332612 953 5...
2585,de7aa2b6adfb9db39489c019f5f3a1a1,3 0.514099 677 1384 1800 1758 0 0.386352 1043 ...


## Merge with Binary classifier output

In [15]:
results_binary_df = pd.read_csv(INPUT_FOLDER_BINARY / "prediction.csv")
display(results_binary_df.head())

results_df = results_yolo_df.merge(results_binary_df, on="image_id")
results_df.loc[results_df["class_name"] == "normal", "PredictionString"] = "14 1 0 0 1 1"
results_df = results_df.drop(columns=["class_name"])

results_df.to_csv(WORK_FOLDER / "submission.csv", index=False)
display(results_df.head())

,image_id,class_name
0,d0615f853a7deeec90f8b1bf30269fcc,normal
1,6637c08da9b3bce162e3aa689da14574,abnormal
2,b65467fc097115261fe11d2a99ccb5cd,abnormal
3,7742abe5bd817ba643d38fb517e35b24,normal
4,2790d36fa3cd4d3fa05229d1c693d499,abnormal


,image_id,PredictionString
0,002a34c58c5b758217ed1f584ccbcfe9,14 1 0 0 1 1
1,004f33259ee4aef671c2b95d54e4be68,0 0.643148 1257 588 1531 874
2,008bdde2af2462e86fd373a445d0f4cd,3 0.477753 1095 1399 1934 1784 0 0.327856 1431...
3,009bc039326338823ca3aa84381f17f1,14 1 0 0 1 1
4,00a2145de1886cb9eb88869c85d74080,3 0.518738 789 1294 1852 1645 0 0.361125 1121 ...


## Submit to Kaggle

In [16]:
submission_message = " ***** ".join([
    version_notes,
    f"Reduce method: {reduce_method}; IoU {iou_threshold}; Conf {skip_box_thrreshold}; Sigma {sigma}",
    f"Binary CLF: {binary_version} -{binary_notes}",
    "Ensemble YOLO: " + "; ".join(f"{yolo_version['version']} - {yolo_version['version_notes']}" for yolo_version in INPUTS_YOLO_VERSIONS),
])
!echo "{submission_message}"
!kaggle competitions submit \
    vinbigdata-chest-xray-abnormalities-detection \
    -f {WORK_FOLDER}/submission.csv \
    -m "{submission_message}"
!sleep 10
!kaggle competitions submissions vinbigdata-chest-xray-abnormalities-detection

BBox reduction ***** Reduce method: weighted_boxes_fusion; IoU 0.35; Conf 0.1; Sigma 0.1 ***** Binary CLF: 4 - EffNet, 1024 resolution, threshold 0.8 ***** Ensemble YOLO: 11 - Remove (most) augs --- WBF preproc IoU: 0.8, version: yolov5l, epochs: 30, valid split: 20.0%, image size: 1024, detect IoU: 0.8, detect conf: 0.05
100% 240k/240k [00:03<00:00, 76.1kB/s]
Successfully submitted to VinBigData Chest X-ray Abnormalities DetectionfileName               date                 description                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              